In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import logging, os,re
import time, dash
from dash import Dash
import plotly.express as px
import plotly.graph_objects as go

In [2]:
raw = pd.read_csv("../DATA/who_mortality_pays.csv", 
                  engine='python',
                  header=None, 
                  dtype=str,
                  sep=';')
header_idx = raw[0].str.startswith("Region Code").idxmax()

df = pd.read_csv("../DATA/who_mortality_pays.csv",
                 engine="python",
                 header=None,
                 dtype=str,
                 sep=';',
                 skiprows=header_idx)
# forcer le split sur les virgules
df = df[0].str.split(",", expand=True)
df.columns = [
    "Region_Code",
    "Region_Name",
    "Country_Code",
    "Country_Name",
    "Year",
    "Sex",
    "Age_group_code",
    "Age_group",
    "Number",
    "Percent_cause_specific_deaths",
    "Age_standardized_rate_100000",
    "Death_rate_per_100000",
    '',
    ""
]
df = df.drop(index=0).reset_index(drop=True)

df.head()


,Region_Code,Region_Name,Country_Code,Country_Name,Year,Sex,Age_group_code,Age_group,Number,Percent_cause_specific_deaths,Age_standardized_rate_100000,Death_rate_per_100000,,
0,EU,Europe,ALB,Albania,1987,All,Age_all,[All],72.00000000,0.41556043,3.66448761,2.34062612,,None
1,EU,Europe,ALB,Albania,1987,All,Age00,[0],0.00000000,0.00000000,,0.00000000,,None
2,EU,Europe,ALB,Albania,1987,All,Age01_04,[1-4],1.00000000,0.08097166,,0.34965035,,None
3,EU,Europe,ALB,Albania,1987,All,Age05_09,[5-9],0.00000000,0.00000000,,0.00000000,,None
4,EU,Europe,ALB,Albania,1987,All,Age10_14,[10-14],0.00000000,0.00000000,,0.00000000,,None


In [3]:
df.dtypes

Region_Code                      object
Region_Name                      object
Country_Code                     object
Country_Name                     object
Year                             object
Sex                              object
Age_group_code                   object
Age_group                        object
Number                           object
Percent_cause_specific_deaths    object
Age_standardized_rate_100000     object
Death_rate_per_100000            object
                                 object
                                 object
dtype: object

In [4]:
num_cols = ["Number", "Age_group_code", "Age_group", "Percent_cause_specific_deaths", "Age_standardized_rate_100000", "Death_rate_per_100000"]

for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")
    
df.head()

,Region_Code,Region_Name,Country_Code,Country_Name,Year,Sex,Age_group_code,Age_group,Number,Percent_cause_specific_deaths,Age_standardized_rate_100000,Death_rate_per_100000,,
0,EU,Europe,ALB,Albania,1987,All,NaN,NaN,72.0,0.415560,3.664488,2.340626,,None
1,EU,Europe,ALB,Albania,1987,All,NaN,NaN,0.0,0.000000,NaN,0.000000,,None
2,EU,Europe,ALB,Albania,1987,All,NaN,NaN,1.0,0.080972,NaN,0.349650,,None
3,EU,Europe,ALB,Albania,1987,All,NaN,NaN,0.0,0.000000,NaN,0.000000,,None
4,EU,Europe,ALB,Albania,1987,All,NaN,NaN,0.0,0.000000,NaN,0.000000,,None


In [11]:
# df['Year'] = pd.to_datetime(df['Year'])
total_2020_2025 = df[(df['Year']>='2019') & (df['Year']<='2025') & (df['Sex']=="Male")]
total_2020_2025

,Region_Code,Region_Name,Country_Code,Country_Name,Year,Sex,Age_group_code,Age_group,Number,Percent_cause_specific_deaths,Age_standardized_rate_100000,Death_rate_per_100000,,
178374,AS,Asia,ARE,United Arab Emirates,2020,Male,NaN,NaN,32.0,0.416667,3.375491,0.468086,,None
178375,AS,Asia,ARE,United Arab Emirates,2020,Male,NaN,NaN,0.0,0.000000,NaN,0.000000,,None
178376,AS,Asia,ARE,United Arab Emirates,2020,Male,NaN,NaN,0.0,0.000000,NaN,0.000000,,None
178377,AS,Asia,ARE,United Arab Emirates,2020,Male,NaN,NaN,0.0,0.000000,NaN,0.000000,,None
178378,AS,Asia,ARE,United Arab Emirates,2020,Male,NaN,NaN,0.0,0.000000,NaN,0.000000,,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210224,CSA,Central and South America,BRA,Brazil,2021,Male,NaN,NaN,2508.0,2.265132,NaN,100.282976,,None
210225,CSA,Central and South America,BRA,Brazil,2021,Male,NaN,NaN,3058.0,2.992524,NaN,193.448008,,None
210226,CSA,Central and South America,BRA,Brazil,2021,Male,NaN,NaN,3079.0,3.365064,NaN,339.921638,,None
210227,CSA,Central and South America,BRA,Brazil,2021,Male,NaN,NaN,4171.0,3.284355,NaN,740.762447,,None


In [17]:
## Enregistrer les données des taux de mortality du cancer dans le monde
total_2020_2025.to_csv("df_mortality_202_2025.csv",columns=['Region_Name', 'Country_Name', "Year", "Sex", "Number", "Percent_cause_specific_deaths", "Death_rate_per_100000"], index=False)


In [19]:
df_2022 = pd.read_csv('/home/donerick/Data-science/DATA/dataset-inc-both-sexes-in-2022-prostate.csv')
df_2022.head(10)

,Alpha-3 code,Cancer code,Population code (ISO/UN),Country,Label,Sex,Number,95% UI low,95% UI high,Number.1,ASR (World),Crude rate,Cumulative risk
0,DZA,27.0,12.0,12.0,Algeria,0.0,3514,3294.0,3749.0,3514,15.5,15.3,1.9
1,AGO,27.0,24.0,24.0,Angola,0.0,2618,2495.0,2747.0,2618,47.9,15.1,5.7
2,BWA,27.0,72.0,72.0,Botswana,0.0,159,118.0,214.0,159,23.9,13.4,2.8
3,BDI,27.0,108.0,108.0,Burundi,0.0,822,608.0,1111.0,822,38.0,13.1,4.5
4,CMR,27.0,120.0,120.0,Cameroon,0.0,2050,1318.0,3189.0,2050,38.6,14.7,4.7
5,CPV,27.0,132.0,132.0,Cape Verde,0.0,77,61.0,98.0,77,43.6,27.0,5.2
6,CAF,27.0,140.0,140.0,Central African Republic,0.0,257,208.0,317.0,257,31.3,10.3,3.6
7,TCD,27.0,148.0,148.0,Chad,0.0,2582,2471.0,2698.0,2582,82.2,29.7,10.6
8,COM,27.0,174.0,174.0,Comoros,0.0,68,58.0,80.0,68,33.2,14.9,3.9
9,COG,27.0,178.0,178.0,"Congo, Republic of",0.0,441,366.0,531.0,441,42.3,15.2,4.8


In [20]:
df_2050 = pd.read_csv("/home/donerick/Data-science/DATA/dataset_2050.csv")
df_2050.head()

,Cancer id,Cancer label,Population id,Population,Sex,Type,Cases base in 2022,Year,Prediction,Change in number of cases,Change in number of cases due to population,Change in number of cases due to risk
0,"27,28",Prostate; Testis,903,Africa,0,0,106189,2022,106189,0,0.0000,0
1,"27,28",Prostate; Testis,903,Africa,0,0,106189,2025,114103,7914,7.4527,0
2,"27,28",Prostate; Testis,903,Africa,0,0,106189,2030,137927,31738,29.8882,0
3,"27,28",Prostate; Testis,903,Africa,0,0,106189,2035,166735,60546,57.0172,0
4,"27,28",Prostate; Testis,903,Africa,0,0,106189,2040,201265,95076,89.5347,0
